<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs
!pip install jupyter-dash
!pip install dash-cytoscape

In [7]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json
import statistics

In [ ]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())
elements = data["nodes"] + data["edges"]
elements[10:40]

In [10]:
graph = {node["data"]["id"]: {"neighbors": list()} for node in data["nodes"]}
for edge in data["edges"]:
  graph[edge["data"]["source"]]["neighbors"].append(edge["data"]["target"])
degrees = []
for node_id, node_content in graph.items():
  node_content['degree'] = len(node_content["neighbors"])
  degrees.append(node_content['degree'])
std_deg, mean_deg, median_deg, max_deg = statistics.stdev(degrees), statistics.mean(degrees), statistics.median(degrees), max(degrees)
high_deg_nodes = [node_id for node_id, node_content in graph.items() if node_content['degree'] > mean_deg + std_deg]
high_deg_nodes

['4162',
 '4160',
 '4158',
 '4150',
 '4148',
 '4140',
 '4134',
 '4132',
 '4113',
 '4111',
 '4105',
 '4097',
 '4093',
 '4088',
 '4085',
 '4083',
 '4081',
 '4079',
 '4077',
 '4075',
 '4073',
 '4071',
 '4066',
 '4064',
 '4062',
 '4060',
 '4058',
 '4054',
 '4052',
 '4050',
 '4048',
 '4046',
 '4044',
 '4042',
 '4040',
 '4038',
 '4036',
 '4031',
 '4030',
 '4028',
 '4026',
 '4024',
 '4022',
 '4018',
 '4016',
 '4010',
 '4006',
 '4004',
 '4002',
 '4000',
 '3996',
 '3994',
 '3992',
 '3990',
 '3988',
 '3978',
 '3976',
 '3972',
 '3967',
 '3957',
 '3955',
 '3944',
 '3942',
 '3938',
 '3931',
 '3929',
 '3927',
 '3923',
 '3919',
 '3912',
 '3908',
 '3906',
 '3900',
 '3896',
 '3894',
 '3890',
 '3882',
 '3880',
 '3878',
 '3876',
 '3872',
 '3868',
 '3866',
 '3864',
 '3862',
 '3860',
 '3856',
 '3854',
 '3850',
 '3848',
 '3844',
 '3840',
 '3838',
 '3832',
 '3830',
 '3828',
 '3826',
 '3824',
 '3822',
 '3820',
 '3818',
 '3810',
 '3870',
 '3808',
 '3806',
 '3802',
 '3800',
 '3798',
 '3792',
 '3790',
 '3788',
 

In [9]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '800px', 'height': '1000px'},
                                      stylesheet = [
        # Group selectors
        {
            'selector': 'node',
            'style': {
                'content': 'data(label)'
            }
        },

        # Class selectors
        {
            'selector': '.inv',
            'style': {
                'background-color': 'red',
                'line-color': 'red'
            }
        },
        {
            'selector': '.non_inv',
            'style': {
                'background-color': 'blue',
                'line-color': 'blue'
            }
        }
    ]
                                      )
                       ])
app.run_server(mode='inline')

<IPython.core.display.Javascript object>